In [1]:
import danra_to_zarr.pipeline.collection
import xarray as xr
from tqdm import tqdm

/home/lcd/git-repos/danra_to_zarr/.venv/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(
/home/lcd/git-repos/danra_to_zarr/.venv/lib/python3.10/site-packages/gribscan/gridutils.py:204: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _calc_lambda_phi_numba(east, north, false_easting, false_northing, r0, theta_0, a, k0, F, n, e):
2024-02-05 18:23:05.824 | INFO     | danra_to_zarr.pipeline.base:<module>:21 - Logging to /nwp/danra/data/v0.3.0/20240205T182305.820797.log
2024-02-05 18:23:05.829 | INFO     | danra_to_zarr.pipeline.collection:<module>:16 - Processing for vers

In [2]:
task = danra_to_zarr.pipeline.collection.DanraCompleteZarrCollection()

In [3]:
task.requires()

{'height_levels': DanraZarrCollection(part_id=height_levels),
 'pressure_levels': DanraZarrCollection(part_id=pressure_levels),
 'single_levels': DanraZarrCollection(part_id=single_levels)}

In [4]:
task1 = task.requires()['height_levels']
task2 = task1.requires()["heightAboveGround"]

In [5]:

datasets = []
inputs = task2.input()
for i, inp in enumerate(tqdm(inputs)):
    ds = inp.open()
    # remove last timestep from all but last dataset
    if i < len(inputs) - 1:
        ds = ds.isel(time=slice(None, -1))
    datasets.append(ds)


/home/lcd/git-repos/danra_to_zarr/.venv/lib/python3.10/site-packages/dmidc/utils.py:19: UserWarning: timestamp has no timezone, assuming system timezone (UTC)
  warnings.warn(
100%|██████████| 269/269 [00:08<00:00, 32.49it/s]


In [6]:

ds = xr.concat(datasets[:10], dim="time")
ds

<xarray.Dataset>
Dimensions:            (y: 589, x: 789, level: 1, time: 1088)
Coordinates:
    forecast_duration  float64 nan
    lat                (y, x) float64 47.65 47.66 47.67 ... 64.42 64.42 64.42
  * level              (level) int64 100
    lon                (y, x) float64 -2.094 -2.064 -2.033 ... 24.29 24.34 24.4
  * time               (time) datetime64[ns] 1990-09-01 ... 1991-01-14T21:00:00
  * x                  (x) float64 -1.999e+06 -1.997e+06 ... -2.925e+04
  * y                  (y) float64 -6.095e+05 -6.07e+05 ... 8.58e+05 8.605e+05
Data variables:
    r                  (level, time, y, x) float64 dask.array<chunksize=(1, 1, 128, 128), meta=np.ndarray>
    t                  (level, time, y, x) float64 dask.array<chunksize=(1, 1, 128, 128), meta=np.ndarray>
    u                  (level, time, y, x) float64 dask.array<chunksize=(1, 1, 128, 128), meta=np.ndarray>
    v                  (level, time, y, x) float64 dask.array<chunksize=(1, 1, 128, 128), meta=np.ndarray>
Attributes:
    centre:             eidb
    centreDescription:  Dublin
    edition:            1
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.Ha...
    subCentre:          255
    suite_name:         DANRA

In [8]:
from pathlib import Path
import shutil

if Path(task2.output().path).exists():
    shutil.rmtree(task2.output().path)

/home/lcd/git-repos/danra_to_zarr/.venv/lib/python3.10/site-packages/dmidc/utils.py:19: UserWarning: timestamp has no timezone, assuming system timezone (UTC)
  warnings.warn(


In [9]:
from dask.distributed import Client

In [10]:
client = Client(n_workers=32, memory_limit='128GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 32
Total threads: 64,Total memory: 3.73 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34237,Workers: 32
Dashboard: http://127.0.0.1:8787/status,Total threads: 64
Started: Just now,Total memory: 3.73 TiB
Comm: tcp://127.0.0.1:40821,Total threads: 2
Dashboard: http://127.0.0.1:34165/status,Memory: 119.21 GiB
Nanny: tcp://127.0.0.1:45671,


In [11]:
self = task2
ds_chunked = ds.chunk(self.rechunk_to)
self.output().write(ds_chunked)

/home/lcd/git-repos/danra_to_zarr/.venv/lib/python3.10/site-packages/dmidc/utils.py:19: UserWarning: timestamp has no timezone, assuming system timezone (UTC)
  warnings.warn(
/home/lcd/git-repos/danra_to_zarr/.venv/lib/python3.10/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 32.62 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
